In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import torch
import tensorflow as tf
import math
from tqdm import tqdm_notebook , tqdm
from collections import deque

# DataLoading

In [14]:

df=pd.read_csv('2011-2019pricedata.csv')#loading Dataset
df=df.dropna() #dropping Nan values
print(df.head())

         Date         Open         High          Low        Close  \
0  2011-01-03  6177.450195  6178.549805  6147.200195  6157.600098   
1  2011-01-04  6172.750000  6181.049805  6124.399902  6146.350098   
2  2011-01-05  6141.350098  6141.350098  6062.350098  6079.799805   
3  2011-01-06  6107.000000  6116.149902  6022.299805  6048.250000   
4  2011-01-07  6030.899902  6051.200195  5883.600098  5904.600098   

     Adj Close  Volume  
0  6157.600098     0.0  
1  6146.350098     0.0  
2  6079.799805     0.0  
3  6048.250000     0.0  
4  5904.600098     0.0  


In [15]:
# opening price would represent the price at a particular day
dataset=list(df['Open'])
len(dataset)

2052

In [16]:
def stock_price_format(n):
    if n < 0 :
        return '-Rs {:2f}'.format(n)
    else :
        return '-Rs {:2f}'.format(n)

# Define State

State would basically include features  like current day price and would also include historical prices which would be used to represent current market conditions

In [17]:
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [18]:
def state_creator(data, timestep, window_size):
    starting_id = timestep - window_size + 1
    if starting_id >= 0:
        windowed_data = list(data[starting_id:timestep+1])
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    state = []
    for i in range(window_size - 1):
        #state would basically consider the increase in prices for the past few days
        #state would also use sigmoid function to recduce the skewness in the increase of price and ouputs between 0 and 1
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    return np.array([state])

# Building the Trader and Model

In [19]:
class ModelandTrade:

    def __init__(self, state_size, action_space=3, model_name="ModelandTrade"):

        self.state_size = state_size
        self.action_space = action_space
        self.model_name = model_name
        self.inventory = []
        self.memory = deque(maxlen=2000)
        #exploration parameters
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995
        #model creation
        self.model = self.model_builder()

    def model_builder(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))
        model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
        #mean Square error
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))

        return model

    def trade(self, state):
        #exploration
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
        #exploitation

        Q_value=self.model.predict(state,verbose=0)
        return np.argmax(Q_value[0])

    def batch_train(self,batch_size):
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])

        for state, action, reward, next_state, done in batch:
            Q_target=reward
            if not done:
                #q_target found using belman equation
                Q_target = reward + self.gamma * np.amax(self.model.predict(next_state,verbose=0)[0])
            target = self.model.predict(state,verbose=0)
            target[0][action] = Q_target
            #fitting the model
            self.model.fit(state, target, epochs=1, verbose=0)
            #Mean Square Error Loss

        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay

## Hyperparmeters

In [20]:
window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(dataset) - 1

In [21]:
trader=ModelandTrade(window_size)

# Reward function

In [22]:
def RewardFunction(action, dataset, datasamples, t, total_profit):
        reward=0
        if action == 1: #Buying
            trader.inventory.append(dataset[t])
            print("AI Trader bought: ", stock_price_format(dataset[t]))

        elif action == 2 and len(trader.inventory) > 0: #Selling
            buy_price = trader.inventory.pop(0)

            reward = max(dataset[t] - buy_price, 0)
            total_profit += dataset[t] - buy_price
            print("AI Trader sold: ", stock_price_format(dataset[t]), " Profit: " + stock_price_format(dataset[t] - buy_price) )

        if t == data_samples - 1:
            done = True
        else:
            done = False

        return reward, done, total_profit


# Training the Dataset

In [ ]:
for episode in range(1, episodes + 1):
    state = state_creator(dataset, 0, window_size + 1)
    total_profit = 0
    trader.inventory = []
    for t in tqdm(range(data_samples)):
        action = trader.trade(state)
        next_state = state_creator(dataset, t+1, window_size + 1)
        reward, done, total_profit = RewardFunction(action, dataset, data_samples, t, total_profit)
        trader.memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print("TOTAL PROFIT: {}".format(total_profit))
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
    if episode % 100 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))


  0%|          | 0/2051 [00:00<?, ?it/s]

AI Trader bought:  -Rs 6177.450195
AI Trader bought:  -Rs 6141.350098
AI Trader sold:  -Rs 6107.000000  Profit: -Rs -70.450195
AI Trader bought:  -Rs 6030.899902
AI Trader bought:  -Rs 5901.299805
AI Trader bought:  -Rs 5767.950195
AI Trader sold:  -Rs 5850.750000  Profit: -Rs -290.600098
AI Trader bought:  -Rs 5648.799805
AI Trader bought:  -Rs 5692.049805
AI Trader sold:  -Rs 5717.100098  Profit: -Rs -313.799804
AI Trader bought:  -Rs 5763.299805
AI Trader sold:  -Rs 5614.000000  Profit: -Rs -287.299805
AI Trader bought:  -Rs 5469.549805
AI Trader bought:  -Rs 5430.450195
AI Trader bought:  -Rs 5519.899902
AI Trader bought:  -Rs 5432.350098
AI Trader sold:  -Rs 5293.049805  Profit: -Rs -474.900390
AI Trader bought:  -Rs 5246.049805
AI Trader bought:  -Rs 5340.250000
AI Trader bought:  -Rs 5467.750000
AI Trader bought:  -Rs 5467.600098
AI Trader sold:  -Rs 5501.700195  Profit: -Rs -147.099610


  2%|▏         | 33/2051 [00:08<09:10,  3.67it/s]

AI Trader sold:  -Rs 5557.549805  Profit: -Rs -134.500000


  2%|▏         | 34/2051 [00:17<21:16,  1.58it/s]

...

 26%|██▌       | 532/2051 [1:06:25<3:00:40,  7.14s/it]

AI Trader bought:  -Rs 5946.100098


 26%|██▌       | 533/2051 [1:06:33<3:05:10,  7.32s/it]

AI Trader sold:  -Rs 5944.600098  Profit: -Rs -1.500000


 26%|██▌       | 534/2051 [1:06:40<2:58:35,  7.06s/it]